## Get the data
csv headings: id, created_at, source, original_text, clean_text, favorite_count, retweet_count, hashtags, trend <br>
hashtags format: strings with comma separated hashtags

In [ ]:
import csv

tweets = []
hashtags = [] #list of lists of hashtags e.g. hashtags[0] = ["hashtag1", "hashtag2"]
hashtags_strings = [] #list of hashtags string e.g. hashtags[0] = ["hashtag1, hashtag2"]

with open('./Data/tweets1.csv') as data_file:
    data = csv.reader(data_file)
    next(data) #To skip the headings
    for row in data:
        tweets.append(row[4])
        hashtags.append(row[7].split(", "))
        hashtags_strings.append(row[7])

## Initialize the tokenizers
Will use a specialized tokenizer for the hashtags because we need to encode all the hashtags. It also does not matter if the encoding of the tweets match the encoding of the hashtags.

In [ ]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

In [ ]:
hashtags_tokenizer = Tokenizer(oov_token="<OOV>")
hashtags_tokenizer.fit_on_texts(hashtags_strings)
hashtags_word_index = hashtags_tokenizer.word_index
hashtags_index_word = hashtags_tokenizer.index_word

## Create the sequences and pad them and one hot encode the hashtags
Will use a binary vector to encode the hashtags to the model can categorize the tweets. e.g. hashtags[0] = [tag1, tag2], and tag1 has encoding of 1 and tag2 has encoding 2, then the binary vector wil be [0 1 1 0 0 ... no_of_different_hashtags]

In [ ]:
sequence_length = 20

from keras.utils import pad_sequences
tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
hashtags_sequences = hashtags_tokenizer.texts_to_sequences(hashtags)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

In [ ]:
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np

no_of_different_hashtags = len(hashtags_word_index) + 1
no_of_hashtags = len(hashtags_sequences)

encoded_hashtags = np.zeros((no_of_hashtags, no_of_different_hashtags))

for i, hashtags_indices in enumerate(hashtags_sequences):
    encoded_hashtags[i][hashtags_indices] = 1


encoded_hashtags = np.array(encoded_hashtags)


## Build the model

In [ ]:
#hyperparameters
embedding_dimensions = 128
lstm_units = 100
dropout_value = 0.2

In [ ]:
no_of_tweets_words = len(tweets_word_index) + 1

hashtag_recommender_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(no_of_tweets_words, embedding_dimensions, input_length=sequence_length),
    tf.keras.layers.Dropout(dropout_value),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_different_hashtags, activation='softmax')
])

hashtag_recommender_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

hashtag_recommender_model.summary()

## Train the model

In [ ]:
epochs = 20
hashtag_recommender_model.fit(tweets_sequences_padded, encoded_hashtags, epochs=epochs)

## Get hashtags!!

In [ ]:
def predict(tweet, tweet_tokenizer, hashtag_tokenizer, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    hashtag_index = np.argmax(prediction, axis=-1)[0]
    return hashtag_tokenizer.index_word[hashtag_index]


In [ ]:
print(predict("happy earth day", tweets_tokenizer, hashtags_tokenizer, 20, hashtag_recommender_model))